In [44]:
import qforte as qf
import numpy as np

$$
\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}
$$

## The FCIComputer class

The `FCIComputer` class is a central class in QForte in that it is used for essentially all applications. The core attributes of this class are the matrix of complex coefficients `C_` stored as a Tensor object, and the corresponding FCIGraph.

The purpose of the `FCIComputer` is to act as a lower memory variant of the regular (sometimes called fock) `Computer` class. The main distinciton is that only coefficients corresponding to a fixed number of alpha and beta (and of course total) electrons are stored. As such the number of electrons, the number of orbitlas (a proxy for qubits), and twice the total spin are specified by the user upon instantiation. 

In a addition to memory savings, the `FCIComputer` (and more specifically the corresponding `FCIGraph`) allow for `SQOperator` application to the state using the Knowls-Handy algorithm, where the coupling coefficients are pre-computed and stored in the `FCIGraph` upon instantion of a new `FCIComputer.` This is generally much faster than than apply Pauli gaues to the regular `Computer` class. Sometimes by a factor of ~500. 

> Instantiate a (Fock) `Computer` with four qubits and print the representation. Note that we always initialize to the vacuum.

> Set that Computer to the HF state asuming two electrons and pring the state (remember only non-zero coefficients are printed).

> Instantiate a `FCIComputer` with two orbitals (corresponding to four qubits), two electonrs, and zero spin. Set the FCIComputer to the Hartree Fock-state. Print the FCIComputer. 

In [28]:
# Start with "Fock" Computer Construction
nqb = 4
fock_comp = qf.Computer(nqb)

X0 = qf.gate('X', 0)
X1 = qf.gate('X', 1)

fock_comp.apply_gate(X0)
fock_comp.apply_gate(X1)

print(fock_comp)

# Now for the equivalent FCIComputer
nel = 2
twos = 0
norb = 2

fci_comp = qf.FCIComputer(
    nel=nel, 
    sz=twos, 
    norb=norb)

fci_comp.hartree_fock()

print(fci_comp)

Computer(
+1.000000 |1100>
)
	
Tensor: FCI Computer
  Ndim  = 2
  Size  = 4
  Shape = (2,2)

  Data:

                   0            1
      0    1.0000000    0.0000000
      1    0.0000000    0.0000000


## Applying Operators to the FCIComputer

An importnat distinction from the regular computer class is that you can only apply number (and spin) presrving second quantized operators (i.e. no Pauli gates or Qubit Operators). Lets take a look at how we would do this.

> Instantiate a 'SQOpertator' that will constitue a linear combination of a single alpa excitation and a double excitation of one alpha electron and one beta electron. 

> Apply that operator first to the regular computer you made above, and then the FCIComputer you made above.

In [29]:
sqop = qf.SQOperator()

h1 = 0.5 
h2 = 0.25
sqop.add_term(h1, [2], [0]) 
sqop.add_term(h2, [3,2], [1,0]) 

print(sqop)

# Apply to Fock Computer
fock_comp.apply_sq_operator(sqop)

# Apply to FCIComputer
fci_comp.apply_sqop(sqop)

# Print both computers and compare the outputs
print(fock_comp)
print(fci_comp)

 +0.500000 ( 2^ 0 )
 +0.250000 ( 3^ 2^ 1 0 )

Computer(
-0.500000 |0110>
-0.250000 |0011>
)
	
Tensor: FCI Computer
  Ndim  = 2
  Size  = 4
  Shape = (2,2)

  Data:

                   0            1
      0    0.0000000    0.0000000
      1    0.5000000   -0.2500000


## Applying Unitary Exponentiated SQOperators to the FCIComputer

As discussed in previous tutorials, we of course want the ability to apply unitaries constructed from exponentiated anti-hermitaion second-quantized operators. Lets consider the follwonig sub example that constructs and apples a small second quantuized operator called K.

> Instantiate a new `FCIComputer` with two orbitals (corresponding to four qubits), two electonrs, and zero spin. Set the FCIComputer to the Hartree Fock-state. Print the FCIComputer. 

> Consturct K, a linear combination of a double excitation and a double de-excitaiotn.

> Apply $\hat{U} = e^{K} = e^{0.5 \hat{a}_2^\dagger \hat{a}_3^\dagger \hat{a}_1 \hat{a}_0 - 0.5 \hat{a}_0^\dagger \hat{a}_1^\dagger \hat{a}_3 \hat{a}_2}$ to the FCIComputer.

> Print the final state of the FCIComputer.

In [32]:

# Define the Computer
nel = 2
twos = 0
norb = 2

fci_comp = qf.FCIComputer(
    nel=nel, 
    sz=twos, 
    norb=norb)

fci_comp.hartree_fock()

print(fci_comp)

# Define K
K = qf.SQOperator()
K.add_term( 0.5, [2,3], [1,0]) 
K.add_term(-0.5, [0,1], [3,2])

# Note that you can multiply K by a time parameter if you want,
# this is helpful for a variety of algorithms, but we can make it 1.0 here.
time = 1.0

# Apply e^time*K to the FCIComputer
fci_comp.apply_sqop_evolution(
time, 
K,
antiherm=True)

print(fci_comp)



	
	
Tensor: FCI Computer
  Ndim  = 2
  Size  = 4
  Shape = (2,2)

  Data:

                   0            1
      0    1.0000000    0.0000000
      1    0.0000000    0.0000000
Tensor: FCI Computer
  Ndim  = 2
  Size  = 4
  Shape = (2,2)

  Data:

                   0            1
      0    0.8775826    0.0000000
      1    0.0000000    0.4794255


## Applying Molecular Hamiltonains to the FCIComputer

Now lets try to do a speed comparison where we look at applying a molecular Hamiltonian. As mentioned in other tutorials the ability to apply the Hamiltonian to an arbitrary state is an important subroutine for many quantum algorithms.

> Create an Berillium Hydride molecue

> Instatiate a FOCK and FCI Computer with the appropriate number of qubits

> Time how long it takes to apply the hamiltonian to one versus the other

> Check how long applicaiton of the Hamiltonian takes using the accelerated `apply_tensor_spat_012bdy()` function which uses the one and two electron integral tensors direction.



In [34]:
# Define the H8 chain geometry
geom = [
    ('H',  (0., 0., 1.0)), 
    ('Be', (0., 0., 2.0)),
    ('H',  (0., 0., 3.0)), 
    ]

# Start a qforte timer
timer = qf.local_timer()

timer.reset()

# Get the molecule object that now contains both the fermionic and qubit Hamiltonians.
mol = qf.system_factory(
    build_type='psi4', 
    mol_geometry=geom, 
    basis='sto-3g', 
    build_qb_ham = False,
    run_fci=1)

# Record how long psi4 took
timer.record('Run Psi4 and Initialize')
 

# Defines the HF state for the Fock Computer
ref = mol.hf_reference
nqb = len(ref)
Uhf = qf.utils.state_prep.build_Uprep(ref, 'occupation_list')


# Define FCIComp parameters
nel = sum(ref)
sz = 0
norb = int(len(ref) / 2)

# Initialize Fock Computer and set to HF state
fock_comp = qf.Computer(nqb)
fock_comp.apply_circuit(Uhf)
 
# Initialize FCIComputer and set to HF state
fci_comp = qf.FCIComputer(
    nel=nel, 
    sz=sz, 
    norb=norb)

fci_comp.hartree_fock()

# Time application of hamiltonian to Fock Computer
timer.reset()
fock_comp.apply_sq_operator(mol.sq_hamiltonian)
timer.record('apply sqop to Fock Computer')

# Time application of hamiltonian to FCI Computer
timer.reset()
fci_comp.apply_sqop(mol.sq_hamiltonian)
timer.record('apply sqop to FCI Computer')

fci_comp.hartree_fock()

# Time application of hamiltonian to FCI Computer
# using accelerated algorithm
timer.reset()
fci_comp.apply_tensor_spat_012bdy(
    mol.nuclear_repulsion_energy, 
    mol.mo_oeis, 
    mol.mo_teis, 
    mol.mo_teis_einsum, 
    norb)

timer.record('apply tensor to FCI Computer')

print(timer)



 ==> Psi4 geometry <==
-------------------------
0  1
H  0.0  0.0  1.0
Be  0.0  0.0  2.0
H  0.0  0.0  3.0
symmetry c1
units angstrom
                Process name                    Time (s)                     Percent
               =============               =============               =============
     Run Psi4 and Initialize                      1.3942                       19.63
 apply sqop to Fock Computer                      5.6996                       80.23
  apply sqop to FCI Computer                      0.0089                        0.13
apply tensor to FCI Computer                      0.0010                        0.01

                  Total Time                      7.1036                      100.00



## Applying Pre-Defined SQOperator "Pools" to a state

It is also helpful to evelove an entire 'pool' of K-typle anti hermitian operatirs, representing a product of unitaries. This is the basic circuit structure for and dUCC type algorithm. An example of how to construct (and evolve a state by) a pool of all particle-hole singles and doubles is shown below. We will contine with some of the the things we defined above.

> Find the energy expectation value of BeH2 for the HF state using the FCIComputer

> Construct a pool of all SD excitations/de-excitations for BeH2 using a pre-defined funciton.

> Apply that pool to the hartree fock sate and find the new energy expecation value

In [43]:
# We want to re-set the state as the HF state first
fci_comp.hartree_fock()

# Get the HF expectaion value for energy
Ehf = fci_comp.get_exp_val(mol.sq_hamiltonian)

# Check that this matches Psi4 (always a good idea)
print(f"Ehf from Psi4:    {mol.hf_energy:8.8f}")
print(f"Ehf from Qforte:  {Ehf:8.8f}")

# Build and fill the pool with particle-hole singles and doubles
pool = qf.SQOpPool()
pool.set_orb_spaces(ref)
pool.fill_pool("SD")

# Apply the pool
fci_comp.evolve_pool_trotter_basic(
            pool,
            antiherm=True,
            adjoint=False)

Enew = fci_comp.get_exp_val(mol.sq_hamiltonian)
print(f"Enew from Qforte: {Enew:8.8f}")


Ehf from Psi4:    -15.45566777
Ehf from Qforte:  -15.45566777+0.00000000j
Enew from Qforte: -7.28864032+0.00000000j


## Apply what you've learned

Now we want to implement dUCCSD-PQE for H2 just like we did at the end of Tutorial 4. But this time use the FCI computer instead of the regular Computer. Template code is provided below, and you should be able to exactly match the numbers from Tutrial 4. 

In [53]:
# Define the reference and geometry lists.
geom = [('H', (0., 0., 0.0)), ('H', (0., 0., 0.75))]

# Get the molecule object that now contains both the fermionic and qubit Hamiltonians.
H2mol = qf.system_factory(
    build_type='psi4', 
    mol_geometry=geom, 
    basis='sto-3g',
    run_fci=1)

# Build the K operator that will perfrom the only viable double excitation/de-excitaiotn for H2

# Define the list for the HF orbital energies.
# Note this is done manually in Tutrial 4, but we can ust grab 
# the pre-computed values from Psi4.
orb_e = []
for i, ei in enumerate(H2mol.hf_orbital_energies):
    orb_e += [ei]*2

# Define the Moller-Plesset denominator ∆_mu.   

print(f"\nDelta_mu:    {delta_mu:+12.10f}")

 ==> Psi4 geometry <==
-------------------------
0  1
H  0.0  0.0  0.0
H  0.0  0.0  0.75
symmetry c1
units angstrom
 +1.000000 ( 2^ 3^ 1 0 )
 -1.000000 ( 0^ 1^ 3 2 )


Delta_mu:    -2.4706932181


In [60]:
def get_energies(t_mu, K_mu, nel, sz, norb):

    # Initialize a FCIComputer and set it to the current state
    # defined by t_mu and K_mu

    # Get the PQE energy of the current state.

    # Get the excited determinannt energy by applying K to the HF state and then applying e^K.
    
    # Reset a FCIComputer to HF State
    
    # Apply the Unitary e^{ (pi/4) Kmu }
    
    # Apply e^K.
    
    
    # Get the mixed state energy
    

    return Eomega, Emu, E0

In [61]:
# Defie the number of PQE iterations and the intial t_mu = 0.0
pqe_iter = 11
t_mu = 0.0

print(f"   Iteration       Epqe                  Emu            E_omega_mu            r_mu")
print(f"--------------------------------------------------------------------------------------------")

# for loop that will print energies at every iteration.
for n in range(pqe_iter):
    
    # return Eomega_mu, Emu, Eo from the funciton you wrote
    
    # define r_mu
    
    # print everyting
    print(f"       {n:2}       {E0:+12.10f}       {Emu:+12.10f}     {Eomega_mu:+12.10f}       {r_mu:+12.10f}")
    
    # Update the amplitude.
    
    
    
print(f'\n\n Efci:   {H2mol.fci_energy:+12.10f}')

   Iteration       Epqe                  Emu            E_omega_mu            r_mu
--------------------------------------------------------------------------------------------
        0       -1.1161514489       +0.4388389026     -0.1568847366       +0.1817715366
        1       -1.1343997677       +0.4570872214     -0.2728388683       +0.0658174049
        2       -1.1367756303       +0.4594630840     -0.3153082198       +0.0233480534
        3       -1.1370743384       +0.4597617921     -0.3303959491       +0.0082603240
        4       -1.1371117228       +0.4597991765     -0.3357348320       +0.0029214411
        5       -1.1371163989       +0.4598038526     -0.3376230863       +0.0010331869
        6       -1.1371169837       +0.4598044374     -0.3382908818       +0.0003653914
        7       -1.1371170569       +0.4598045106     -0.3385270509       +0.0001292223
        8       -1.1371170660       +0.4598045197     -0.3386105731       +0.0000457000
        9       -1.1371170672   